In [ ]:
!pip -q install -U \
  langchain langchain-community langchain-text-splitters \
  pypdf sentence-transformers chromadb


In [ ]:
import os

os.listdir("/content")

['.config',
 'sample_data',
 'National_Building_Code_of_Nigeria_2006.pdf',
 'chroma_civil_db']

In [ ]:
import os

# Find the PDF automatically (so your code doesn't break if the filename changes)
pdf_files = [f for f in os.listdir("/content") if f.lower().endswith(".pdf")]
print("PDFs found:", pdf_files)

assert len(pdf_files) > 0, "No PDF found in /content. Upload it first."

pdf_path = os.path.join("/content", pdf_files[0])
print("Using PDF:", pdf_path)


PDFs found: ['National_Building_Code_of_Nigeria_2006.pdf']
Using PDF: /content/National_Building_Code_of_Nigeria_2006.pdf


In [ ]:
import sys
!{sys.executable} -m pip install -q langchain-community pypdf

from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(pdf_path)
raw_docs = loader.load()

print("Loaded pages:", len(raw_docs))
print("Sample page metadata:", raw_docs[0].metadata)
print("Sample text preview:\n", raw_docs[0].page_content[:400])

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Loaded pages: 476
Sample page metadata: {'producer': 'Microsoft® Word 2010', 'creator': 'Microsoft® Word 2010', 'creationdate': '2013-01-15T17:55:12+01:00', 'title': 'untitled', 'author': 'EWULE FAMILY', 'moddate': '2013-01-15T17:55:12+01:00', 'source': '/content/National_Building_Code_of_Nigeria_2006.pdf', 'total_pages': 476, 'page': 0, 'page_label': '1'}
Sample text preview:
 NATIONAL BUILDING 
CODE


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Split pages into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150,
    separators=["\n\n", "\n", " ", ""]
)

docs = text_splitter.split_documents(raw_docs)

print("Total chunks:", len(docs))
print("Sample chunk metadata:", docs[0].metadata)
print("Sample chunk preview:\n", docs[0].page_content[:500])


Total chunks: 1595
Sample chunk metadata: {'producer': 'Microsoft® Word 2010', 'creator': 'Microsoft® Word 2010', 'creationdate': '2013-01-15T17:55:12+01:00', 'title': 'untitled', 'author': 'EWULE FAMILY', 'moddate': '2013-01-15T17:55:12+01:00', 'source': '/content/National_Building_Code_of_Nigeria_2006.pdf', 'total_pages': 476, 'page': 0, 'page_label': '1'}
Sample chunk preview:
 NATIONAL BUILDING 
CODE


In [ ]:
!pip install -q chromadb # Added to install missing dependency

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# Free embeddings model (no API key)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create Chroma DB (stored locally inside Colab session)
vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory="chroma_civil_db"
)

vectordb.persist()
print("Vector DB built and saved to: chroma_civil_db")

/tmp/ipython-input-1507599550.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/tr

In [ ]:
import os
os.environ["HF_HUB_DISABLE_TELEMETRY"] = "1"
os.environ["HF_HUB_DISABLE_IMPLICIT_TOKEN"] = "1"   # key line


In [ ]:
!pip -q install -U langchain langchain-community langchain-text-splitters langchain-huggingface chromadb pypdf sentence-transformers


In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

pdf_path = "/content/National_Building_Code_of_Nigeria_2006.pdf"

loader = PyPDFLoader(pdf_path)
raw_docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=120)
docs = splitter.split_documents(raw_docs)

print("Pages:", len(raw_docs))
print("Chunks:", len(docs))


Pages: 476
Chunks: 1583


In [ ]:
import os

persist_dir = "chroma_civil_db"

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

if os.path.exists(persist_dir) and len(os.listdir(persist_dir)) > 0:
    vectordb = Chroma(persist_directory=persist_dir, embedding_function=embeddings)
    print("Loaded existing Vector DB ✅")
else:
    vectordb = Chroma.from_documents(docs, embedding=embeddings, persist_directory=persist_dir)
    vectordb.persist()
    print("Built + saved new Vector DB ✅")


Loaded existing Vector DB ✅


/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/tmp/ipython-input-1749189218.py:8: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=persist_dir, embedding_function=embeddings)


In [ ]:
results = vectordb.similarity_search("foundation requirements", k=3)

for i, r in enumerate(results, 1):
    print(f"\n--- Result {i} ---")
    print("Page:", r.metadata.get("page"))
    print(r.page_content[:300])



--- Result 1 ---
Page: 455
FOUNDATIONS/BASEMENT COMPLIANCE FORM — continued

--- Result 2 ---
Page: 455
FOUNDATIONS/BASEMENT COMPLIANCE FORM — continued

--- Result 3 ---
Page: 455
FOUNDATIONS/BASEMENT COMPLIANCE FORM — continued


In [ ]:
!pip -q install -U transformers tokenizers accelerate sentencepiece


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 45.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 1.2.0 requires huggingface-hub<1.0.0,>=0.33.4, but you have huggingface-hub 1.4.1 which is incompatible.


In [ ]:
!pip -q install -U "transformers>=4.38" "tokenizers>=0.15" accelerate sentencepiece


In [ ]:
import transformers, tokenizers
print("transformers:", transformers.__version__)
print("tokenizers:", tokenizers.__version__)


In [ ]:
!pip -q install -U "transformers==4.44.2" "tokenizers==0.19.1" accelerate sentencepiece


In [ ]:
import transformers, tokenizers
print("transformers:", transformers.__version__)
print("tokenizers:", tokenizers.__version__)


transformers: 4.44.2
tokenizers: 0.19.1


In [ ]:
import transformers, tokenizers, langchain, chromadb

print("transformers:", transformers.__version__)
print("tokenizers:", tokenizers.__version__)
print("langchain:", langchain.__version__)
print("chromadb:", chromadb.__version__)


transformers: 4.44.2
tokenizers: 0.19.1
langchain: 1.2.9
chromadb: 1.4.1


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

model_name = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

llm = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=300
)


In [ ]:
def ask_smart_civil_engineer(question):
    # NEW: retriever uses invoke()
    docs = retriever.invoke(question)

    context = "\n\n".join(
        [f"(Page {d.metadata.get('page')}): {d.page_content}" for d in docs]
    )

    prompt = f"""
You are a civil engineering assistant.
Answer ONLY using the context below.
If the answer is not found, say "Not specified in the code".

Context:
{context}

Question:
{question}

Answer:
"""

    response = llm(prompt)[0]["generated_text"]
    pages = sorted(set(d.metadata.get("page") for d in docs if d.metadata.get("page") is not None))
    return response, pages


In [ ]:
answer, pages = ask_smart_civil_engineer("What does the code say about foundation requirements?")
print("ANSWER:\n", answer)
print("\nCITED PAGES:", pages)


Token indices sequence length is longer than the specified maximum sequence length for this model (1343 > 512). Running this sequence through the model will result in indexing errors


ANSWER:
 Not specified in the code

CITED PAGES: [23, 249, 420, 445, 448, 449, 453, 455]


In [ ]:
docs = vectordb.similarity_search("foundation requirements", k=6)
for i, d in enumerate(docs, 1):
    print(f"\n--- RESULT {i} | Page: {d.metadata.get('page')} ---")
    print(d.page_content[:800])



--- RESULT 1 | Page: 455 ---
FOUNDATIONS/BASEMENT COMPLIANCE FORM — continued

--- RESULT 2 | Page: 455 ---
FOUNDATIONS/BASEMENT COMPLIANCE FORM — continued

--- RESULT 3 | Page: 455 ---
FOUNDATIONS/BASEMENT COMPLIANCE FORM — continued

--- RESULT 4 | Page: 455 ---
FOUNDATIONS/BASEMENT COMPLIANCE FORM — continued

--- RESULT 5 | Page: 453 ---
FOUNDATIONS/BASEMENT COMPLIANCE FORM — continued 
 
 
465 
NATIONAL BUILDING CODE 
 
Registered Architect 
I certify that I supervised this stage of work and that the work is in accordance with the approved design and specifications. 
Name/Registration No of Architect Signature/ARCON Seal Date 
NB: This should be produced and signed in 5 copies. 
Number of items covered shall be taken as minimum required depending on the scale and complexity of project. 
 ______ ROOFING AND CLOSING - UP COMPLIANCE FORM 
Project title 
Name of owner 
Site location Plot No. 
Operations 
Wall plate (where applicable) Material used Adequacy in size Adequacy in streng

In [ ]:
retriever = vectordb.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 8, "fetch_k": 40, "lambda_mult": 0.6}
)


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""]
)

docs = text_splitter.split_documents(raw_docs)
print("Chunks:", len(docs))


Chunks: 1101


In [1]:
import os, shutil

PERSIST_DIR = "/content/chroma_civil_db"  # THIS is writable in Colab

# Delete old db folder if it exists (it can be locked/corrupted)
if os.path.exists(PERSIST_DIR):
    shutil.rmtree(PERSIST_DIR)

os.makedirs(PERSIST_DIR, exist_ok=True)
print("DB folder ready:", PERSIST_DIR)


DB folder ready: /content/chroma_civil_db


In [ ]:
Smart Civil Engineer – RAG System

• Built a Retrieval-Augmented Generation system over the Nigerian National Building Code
• Uses HuggingFace embeddings + ChromaDB
• Answers civil engineering compliance questions with page citations
• No paid APIs used

Tech: Python, LangChain, ChromaDB, HuggingFace, Google Colab
